In [ ]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI, api_key
import requests
import pdfplumber
from io import BytesIO

# Restaurant Menu Analysis Tool

## Overview
This notebook demonstrates how to use **AI (OpenAI API)** to intelligently analyze restaurant menus. Specifically, it:
- Extracts text content from a PDF menu
- Uses GPT-4 to understand and analyze the menu
- Answers customer questions about menu items (e.g., "What's best for vegetarians?")

## Workflow
1. **Load API credentials** - Initialize OpenAI API connection
2. **Extract menu** - Download and extract text from PDF menu
3. **Process with AI** - Send menu to GPT-4 with a specific question
4. **Display results** - Show the AI's response in a readable format

This is useful for restaurants that want to help customers quickly find menu options matching their dietary preferences.

In [ ]:
# Load environment variables from .env file
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")
elif not api_key.startswith('sk-proj'):
    raise ValueError("Invalid API key format. Please ensure your API key starts with 'sk-proj'.")
elif api_key.strip() != api_key:
    raise ValueError("API key contains leading or trailing whitespace. Please remove any extra spaces.")
else:
    print("API key is valid and properly formatted.")

In [ ]:
#Define the system prompt for summarizing restaurant menus
system_prompt = """You are a helpful assistant that summarizes restaurant menus in spanish. You will be given a full menu of a restaurant, and your task is to extract the key information about the dishes, including their names, descriptions, and prices. Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown. Do not add any highlighting or formatting beyond what is necessary for clarity. Focus on providing a useful summary that helps customers make informed decisions about what to order."""

In [ ]:
#Define our user prompt

user_prompt_prefix = """Here is the menu of a restaurant. Which is the best option for a vegetarian person ? \n\n
"""

In [ ]:
# Define a function to extract text from a PDF menu given its URL. This function will download the PDF, extract the text using pdfplumber, and return the combined text from the specified number of pages.
def extract_menu_text_from_pdf(url):
    # Download the PDF file from the URL
    response = requests.get(url)
    pdf_file = BytesIO(response.content)

    # Extract text from the PDF using pdfplumber
    menu_text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page_num, page in enumerate(pdf.pages):  # Limit to the specified number of pages
            text = page.extract_text()
            menu_text += f"Page {page_num + 1}:\n{text}\n"
    return menu_text

In [ ]:
# Define a function to create the messages for the OpenAI API call, combining the system prompt and the user prompt with the menu text.
def messages_for(menu_text):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + menu_text}
    ]

In [ ]:
# Define the main function to summarize the menu. This function will call the text extraction function, create the messages for the API call, and then use the OpenAI client to get the summary of the menu.
def summarize_menu(menu_url):
    # Extract the menu text from the PDF
    menu_text = extract_menu_text_from_pdf(menu_url)

    # Create the messages for the OpenAI API call
    messages = messages_for(menu_text)

    # Initialize the OpenAI client and make the API call to get the summary
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )

    # Extract and return the summary from the response
    summary = response.choices[0].message.content
    return summary

In [ ]:
# Define a function to display the summary in a Markdown format. This function will call the summarize_menu function and then use IPython's display function to show the summary in a readable format.
def display_summary(url):
    summary = summarize_menu(url)
    display(Markdown(summary))


In [ ]:
# Now we can call the display_summary function with the URL of the PDF menu to see the results.
url = "https://us-mia-1.linodeobjects.com/crepes-main/menublock/b42a132658814753acb7b5c2bd32cda5/af-menu-pag-web-interior-2.pdf"
display_summary(url)